In [10]:
import random
import math
import gc

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch
from torch_geometric.data import Data

tqdm.pandas()

In [11]:
n_samples = 1000
min_nodes = 4
max_nodes = 20
min_edges = 4

data_list = []

for i in tqdm(range(n_samples)):
    n_nodes = random.randint(min_nodes, max_nodes)
    x_features = torch.randn((n_nodes, 3))
    x_mask = torch.zeros((n_nodes, 1))
    x = torch.cat([x_features, x_mask], dim=1)
        
#     max_edges = math.factorial(n_nodes)
    n_edges = random.randint(min_edges, n_nodes * 2)
    edge_index = torch.randint(0, n_nodes - 1, (2, n_edges)).long()
    
    direction = torch.tensor([[int(edge_index[0][i] > edge_index[1][i])] for i in range(n_edges)])
    
    edge_attr = torch.cat([torch.randn((n_edges, 1)), direction, torch.zeros((n_edges, 2))], dim=1)

    data = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr
    )
    
    data_list.append(data)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
data_list[0].x.shape[1]

4

In [13]:
data_list[0].edge_attr.shape[1]

4

In [14]:
CONFIG = {
    "device": 0,
    "batch_size": 256,
    "epochs": 100,
    "lr": 0.001,
    "decay": 0,
    "num_layer": 5,
    "emb_dim": 300,
    "dropout_ratio": 0,
    "mask_rate": 0.15,
    "mask_edge": 1,
    "JK": "last",
    "output_model_file": '',
    "gnn_type": "gat",
    "seed": 0, 
    "num_workers": 8
}

In [15]:
CONFIG["num_layer"]

5

In [16]:
def compute_accuracy(pred, target):
    # return float(torch.sum((pred.detach() > 0) == target.to(torch.uint8)).cpu().item())/(pred.shape[0]*pred.shape[1])
    return float(torch.sum(torch.max(pred.detach(), dim=1)[1] == target).cpu().item()) / len(pred)


def train(config, model_list, loader, optimizer_list, device):
    model, linear_pred_nodes, linear_pred_edges = model_list
    optimizer_model, optimizer_linear_pred_nodes, optimizer_linear_pred_edges = optimizer_list

    model.train()
    linear_pred_nodes.train()
    linear_pred_edges.train()

    loss_accum = 0
    n_broken_batches = 0
    
    pbar = tqdm(loader, desc="Iteration")
    for step, batch in enumerate(pbar):
        batch = batch.to(device)
        
        optimizer_model.zero_grad()
        optimizer_linear_pred_nodes.zero_grad()
        optimizer_linear_pred_edges.zero_grad()
                
        node_rep = model(batch.x, batch.edge_index, batch.edge_attr)

        ## loss for nodes
        pred_node = linear_pred_nodes(node_rep[batch.masked_atom_indices])
        node_loss = criterion(pred_node.double()[:, :3].double(), batch.mask_node_label[:, :3].double()) # FIX_HARDCODE: change [:, 0] to [:, edge_features]
        # ADD: Computation of CrossEntropy for direction besides MSE

#         if config["mask_edge"]:
        masked_edge_index = batch.edge_index[:, batch.connected_edge_indices]
        edge_rep = node_rep[masked_edge_index[0]] + node_rep[masked_edge_index[1]]
        pred_edge = linear_pred_edges(edge_rep)
        edge_loss = criterion(pred_edge[:, 0].double(), batch.mask_edge_label[:, 0].double())
        
        
        loss = node_loss + edge_loss
        clip_loss = loss if not torch.isnan(loss).item() else torch.tensor(0.)
        
        if torch.isnan(clip_loss).item():
            pbar.set_description(f"Loss broken at batch {step}")
            
        if not torch.isnan(loss).item():
            clip_loss.backward() 
        
            
        optimizer_model.step()
        optimizer_linear_pred_nodes.step()
        optimizer_linear_pred_edges.step()

        loss_accum += float(clip_loss.cpu().item())

    return loss_accum / (step - n_broken_batches + 1)


In [ ]:
import argparse

from chem.dataloader import DataLoaderMasking  # , DataListLoader
from chem.mydataset import MyDataset

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np

from chem.model import GNN, GNN_graphpred

import pandas as pd

from chem.util import MaskNode

from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool

criterion = nn.MSELoss()

import timeit
import warnings 
warnings.filterwarnings("ignore")

torch.manual_seed(0)
np.random.seed(0)
device = torch.device("cuda:" + str(CONFIG["device"])) if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

print("num layer: %d mask rate: %f mask edge: %d" %(CONFIG["num_layer"], CONFIG["mask_rate"], CONFIG["mask_edge"]))

dataset = MyDataset(data_list, transform=MaskNode(num_node_features=data_list[0].x.shape[1],
                                                  num_edge_features=data_list[0].edge_attr.shape[1],
                                                  mask_rate=CONFIG["mask_rate"], 
                                                  mask_edge=CONFIG["mask_edge"]))

loader = DataLoaderMasking(dataset, batch_size=1, shuffle=True, num_workers=0)

model = GNN(CONFIG["num_layer"], 
            CONFIG["emb_dim"], 
            x_input_dim = data_list[0].x.shape[1],
            edge_attr_input_dim = data_list[0].edge_attr.shape[1],
            JK = CONFIG["JK"], 
            drop_ratio = CONFIG["dropout_ratio"], 
            gnn_type = CONFIG["gnn_type"]).to(device)
linear_pred_nodes = torch.nn.Linear(CONFIG["emb_dim"], data_list[0].x.shape[1]).to(device)
linear_pred_edges = torch.nn.Linear(CONFIG["emb_dim"], data_list[0].edge_attr.shape[1] - 2).to(device)

model_list = [model, linear_pred_nodes, linear_pred_edges]

# set up optimizers
optimizer_model = optim.Adam(model.parameters(), lr=CONFIG["lr"], weight_decay=CONFIG["decay"])
optimizer_linear_pred_nodes = optim.Adam(linear_pred_nodes.parameters(), lr=CONFIG["lr"], weight_decay=CONFIG["decay"])
optimizer_linear_pred_edges = optim.Adam(linear_pred_edges.parameters(), lr=CONFIG["lr"], weight_decay=CONFIG["decay"])

optimizer_list = [optimizer_model, optimizer_linear_pred_nodes, optimizer_linear_pred_edges]

for epoch in range(1, CONFIG["epochs"] + 1):
    print("====epoch " + str(epoch))

    train_loss = train(CONFIG, model_list, loader, optimizer_list, device)
    print(train_loss)

if not CONFIG["model_file"] == "":
    torch.save(model.state_dict(), CONFIG["model_file"] + ".pth")

num layer: 5 mask rate: 0.150000 mask edge: 1
====epoch 1


Iteration: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 101.34it/s]


2.336591654641272
====epoch 2


Iteration: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 99.98it/s]


1.4817203775001981
====epoch 3


...

Iteration: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 63.75it/s]


0.0025653951540240195
====epoch 34


Iteration: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 63.78it/s]


0.0006536351096098714
====epoch 35


Iteration:  37%|████████████████████████████████████████████▎                                                                            | 366/1000 [00:06<00:08, 74.30it/s]

In [162]:
import importlib
from chem import model, util, batch

importlib.reload(model)
importlib.reload(util)
importlib.reload(batch)

<module 'chem.batch' from '/Users/user/PycharmProjects/pretrain_gnns/chem/batch.py'>

In [ ]:
!cat "chem/batch.py"